## Capstone Check-in #2 
Dustin Stewart  
DSIR 2-8

**Problem:**   
Due to the unpredictable and expansive nature of the sport, it remains hard for football clubs to properly assess the value of prospective signings. Much of the previous techniques relied on scouting players in person and mostly semi-educated gambling. The result is an enormous amount of poor performing signings in the football world. In this project, I will develop a machine learning model that will predict a players average goals and assists per game over the next 3 years using aggregated historical time series data as well as global data. Those predictions can then be used to find the highest performing players relative to their estimated value. For example, you could get a list of players who are predicted to average more than 0.5 goals a match and then rank them by ascending estimated market value (cheapest first). You would then look to purchase the cheaper players or further scout them in person.

**Data Collection:**  
Collected from TransferMarkt, one of the most extensive online collections of football stats. I scraped using a function that navigates to each player from the top 10 leagues in Europe and scrapes their [full stats](https://www.transfermarkt.us/sadio-mane/leistungsdatendetails/spieler/200512/plus/0?saison=&verein=&liga=&wettbewerb=&pos=&trainer_id=).

**EDA Methods:**  
Considering using ratios for values. If a player scores 5 goals in 3 matches, that's much better than somebody who scores 6 goals in 40 matches.

We're looking at players who are 20-26 in 2016, because we're only interested in younger players who are less injury prone and will give us value longer and we can resell at their peak for a profit. 

I will be removing all players with very little playing data. If they don't have senior playing experience, then we're not interested in those players. At least not at this point. I'd need to gather data that isn't easily available and is not related to the purpose of this project.

All players who don't play in attacking positions won't be used. I want to predict attacking player performance for the purpose of this project and to keep a reasonable scope.

**Risks, Assumptions, General Concerns Going into the Future:**  
It's not clear that this data will be enough to make accurate predictions even if it's aggregated. Football is an inherently unpredictable game and stats don't tell as much as most other sports. That being said, sometimes there's patterns that human minds cannot comprehend and that's the goal for the project. Also, there's just many factors that can't be taken into account by data. This is why football has been reluctant to adopt data-centric player recruitment. 

**Scraping function that isn't polished yet:**

In [ ]:
home_request = requests.get('https://www.transfermarkt.us/wettbewerbe/europa', headers={'User-Agent': 'Custom5'})
home_html = home_request.text
home_soup = BeautifulSoup(home_html, 'lxml')
leagues_table = home_soup.find_all('table')
leagues_table = leagues_table[0].find_all('a')[4:60]
full_list = []
league_urls = []
for string in leagues_table:
    if 'startseite' in string['href'] and '1' in string['href'] and string['href'][-2] != '+':
        league_urls.append(string['href'])
for league_url in league_urls[8:9]:
    league_dict = {}
    
    league_name = league_url[1:league_url.find('/startseite')]
    home_request = requests.get('https://www.transfermarkt.us' + league_url + '/plus/?saison_id=2010', headers={'User-Agent': 'Custom5'})
    home_html = home_request.text
    home_soup = BeautifulSoup(home_html, 'lxml')
    clubs_table = home_soup.find_all('table')[3]
    clubs_table_tags = clubs_table.find_all('a')
    club_urls = []
    for string in clubs_table_tags:
        if 'startseite' in string['href'] and 'saison' in string['href'] and string['href'] not in club_urls:
            club_urls.append(string['href'])
    for club_url in club_urls[10:11]:
        print(club_url)
        print(len(club_urls))
        club_dict = {}
        
        club_name = club_url[1:club_url.find('/startseite')]
        
        
        
        home_request = requests.get('https://www.transfermarkt.us' + club_url[:-4] + '2010', headers={'User-Agent': 'Custom5'})
        home_html = home_request.text
        home_soup = BeautifulSoup(home_html, 'lxml')
        players_table = home_soup.find_all('table')[1]
        players_table_tags = players_table.find_all('a')
        player_urls = []
        for string in players_table_tags:
            if 'spieler' in string['href'] and 'profil' in string['href'] and string['href'] not in player_urls:
                player_urls.append(string['href'])
        for player_url in player_urls[:]:
            
            
            
            player_dict = {}
            
            url1 = 'http://www.transfermarkt.us' + player_url[0:-21] + '/leistungsdatendetails/' + player_url[-13:] + '/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'
            url2 = 'http://www.transfermarkt.us' + player_url[0:-22] + '/leistungsdatendetails/' + player_url[-14:] + '/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'
            url3 = 'http://www.transfermarkt.us' + player_url[0:-20] + '/leistungsdatendetails/' + player_url[-12:] + '/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'
            try:
    
                
                if url1.count('/') == 20:

                    print('url1:')
                    print(url1)
                    player_name = url1[28:-98].replace('-', '_')

                    home_request = requests.get(url1, headers={'User-Agent': 'Custom5'})
                    home_html = home_request.text
                    home_soup = BeautifulSoup(home_html, 'lxml')
                    player_table = home_soup.find_all('table', {"class": 'items'})[0]

                    player_clubs = player_table.find_all("a", {"class": 'vereinprofil_tooltip'})
                    club_column = []
                    for club in player_clubs:
                        
                        club_column.append(club['href'][1:club['href'].find('/startseite')])
                    df = pd.read_html(str(player_table))

                    df = df[0]


                    df.drop(df.columns[-1], axis=1, inplace = True)

                    df.drop(index = len(df)-1, inplace = True)

                    df.drop(df.columns[9:-1], axis=1, inplace = True)
                    
                    df.drop(df.columns[1], axis=1, inplace = True)
                    df.drop(df.columns[2], axis=1, inplace = True)
                    column_names = ['season', 'competition', 'in_squad', 'appearances', 'points_per_match', 'goals', 'assists', 'minutes_played']
                    to_rename = dict(zip(df.columns, column_names))

                    df.rename(columns = to_rename, inplace = True)
                    df['club'] = club_column
                    player_dict[url1[28:-98].replace('-', '_') + ', ' + club_name + ', ' + league_name] = df
                    full_list.append(player_dict)

                elif url2.count('/') == 20:
                    print('url2:')
                    print(url2)
                    player_name = url1[28:-99].replace('-', '_')


                    home_request = requests.get(url2, headers={'User-Agent': 'Custom5'})
                    home_html = home_request.text
                    home_soup = BeautifulSoup(home_html, 'lxml')
                    player_table = home_soup.find_all('table', {"class": 'items'})[0]
                    player_clubs = player_table.find_all("a", {"class": 'vereinprofil_tooltip'})
                    club_column = []
                    for club in player_clubs:
                        
                        club_column.append(club['href'][1:club['href'].find('/startseite')])
                    df = pd.read_html(str(player_table))

                    df = df[0]


                    df.drop(df.columns[-1], axis=1, inplace = True)

                    df.drop(index = len(df)-1, inplace = True)

                    df.drop(df.columns[9:-1], axis=1, inplace = True)
                    
                    df.drop(df.columns[1], axis=1, inplace = True)
                    df.drop(df.columns[2], axis=1, inplace = True)
                    column_names = ['season', 'competition', 'in_squad', 'appearances', 'points_per_match', 'goals', 'assists', 'minutes_played']
                    to_rename = dict(zip(df.columns, column_names))

                    df.rename(columns = to_rename, inplace = True)
                    df['club'] = club_column
                    player_dict[url2[28:-99].replace('-', '_') + ', ' + club_name + ', ' + league_name] = df
                    full_list.append(player_dict)


                elif url3.count('/') == 20:
                    print('url3:')
                    print(url3)
                    player_name = url3[28:-97].replace('-', '_')

                    home_request = requests.get(url3, headers={'User-Agent': 'Custom5'})
                    home_html = home_request.text
                    home_soup = BeautifulSoup(home_html, 'lxml')
                    player_table = home_soup.find_all('table', {"class": 'items'})[0]
                    player_clubs = player_table.find_all("a", {"class": 'vereinprofil_tooltip'})
                    club_column = []
                    for club in player_clubs:
                        
                        club_column.append(club['href'][1:club['href'].find('/startseite')])
                    df = pd.read_html(str(player_table))

                    df = df[0]


                    df.drop(df.columns[-1], axis=1, inplace = True)

                    df.drop(index = len(df)-1, inplace = True)

                    df.drop(df.columns[9:-1], axis=1, inplace = True)
                    
                    df.drop(df.columns[1], axis=1, inplace = True)
                    df.drop(df.columns[2], axis=1, inplace = True)
                    column_names = ['season', 'competition', 'in_squad', 'appearances', 'points_per_match', 'goals', 'assists', 'minutes_played']
                    to_rename = dict(zip(df.columns, column_names))

                    df.rename(columns = to_rename, inplace = True)
                    df['club'] = club_column
                    player_dict[url3[28:-97].replace('-', '_')] = df
                    full_list.append(player_dict)

            except:
                continue 
len(full_list)        